![Microsoft](https://raw.githubusercontent.com/microsoft/azuredatastudio/main/extensions/resource-deployment/images/microsoft-small-logo.png)
## Deploy Azure SQL Edge container instance locally with Docker
This notebook will use Docker to pull and run the Azure SQL Edge image and connect to it in Azure Data Studio

### Dependencies
- Docker Engine. For more information, see [Install Docker](https://docs.docker.com/engine/installation/).

<span style="color:red"><font size="3">Please press the "Run all" button to run the notebook</font></span>

### Check dependencies

In [ ]:
import pandas,sys,os,getpass,json,html,time
from string import Template
pandas_version = pandas.__version__.split('.')
pandas_major = int(pandas_version[0])
pandas_minor = int(pandas_version[1])
pandas_patch = int(pandas_version[2])
if not (pandas_major > 0 or (pandas_major == 0 and pandas_minor > 24) or (pandas_major == 0 and pandas_minor == 24 and pandas_patch >= 2)):
    sys.exit('Please upgrade the Notebook dependency before you can proceed, you can do it by running the "Reinstall Notebook dependencies" command in command palette (View menu -> Command Palette…).')

def run_command(displayCommand = ""):
    print("Executing: " + (displayCommand if displayCommand != "" else cmd))
    !{cmd}
    if _exit_code != 0:
        sys.exit(f'Command execution failed with exit code: {str(_exit_code)}.\n')
    print(f'Command successfully executed')

cmd = 'docker version'
run_command()

### List existing containers
You can view the ports that have been used by existing containers

In [ ]:
cmd = f'docker ps -a'
run_command()

### Setting variables

In [ ]:
env_var_flag = "AZDATA_NB_VAR_SA_PASSWORD" in os.environ
password_name = 'SQL Server sa account password'
if env_var_flag:
    container_name = os.environ["AZDATA_NB_VAR_DOCKER_CONTAINER_NAME"]
    sql_password = os.environ["AZDATA_NB_VAR_SA_PASSWORD"]
    sql_port = os.environ["AZDATA_NB_VAR_DOCKER_PORT"]
    docker_registry = os.environ["AZDATA_NB_VAR_DOCKER_REGISTRY"]
    docker_repository = os.environ["AZDATA_NB_VAR_DOCKER_REPOSITORY"]
    docker_imagetag = os.environ["AZDATA_NB_VAR_DOCKER_IMAGETAG"]
    docker_username = os.environ["AZDATA_NB_VAR_DOCKER_USERNAME"]
    docker_password = os.environ["AZDATA_NB_VAR_DOCKER_PASSWORD"]
else:
    container_name = 'SQLEDGE-' + time.strftime("%Y%m%d%H%M%S", time.localtime())
    docker_registry = 'mcr.microsoft.com'
    docker_repository = 'azure-sql-edge-developer'
    docker_imagetag = 'latest'
    docker_username = ''
    docker_password = ''
    sql_password = getpass.getpass(prompt = password_name)
    password_confirm = getpass.getpass(prompt = f'Confirm {password_name}')
    if sql_password != password_confirm:
        sys.exit(f'{password_name} does not match the confirmation password.')
    sql_port = input('SQL Server port, default value is 1433')
    if len(sql_port) == 0:
        sql_port = '1433'
print(f'{password_name}: ******')
print(f'Container name: {container_name}')
print(f'Port: {sql_port}')
print(f'Docker registry: {docker_registry}')
print(f'Docker repository: {docker_repository}')
print(f'Image tag: {docker_imagetag}')

### Pull the container image

In [ ]:
if docker_username != '':
    cmd = f'docker login {docker_registry} -u {docker_username} -p {docker_password}'
    run_command(False)
cmd = f'docker pull {docker_registry}/{docker_repository}:{docker_imagetag}'
run_command()

### Start a new container

In [ ]:
template = Template(f'docker run -e ACCEPT_EULA=Y -e "SA_PASSWORD=$password" -p {sql_port}:1433 --name {container_name} -d {docker_registry}/{docker_repository}:{docker_imagetag}')
cmd = template.substitute(password=sql_password)
run_command(template.substitute(password='******'))

### List all the containers

In [ ]:
cmd = f'docker ps -a'
run_command()

### Connect to Azure SQL Edge instance in Azure Data Studio
It might take a couple minutes for the service to launch

In [ ]:
from IPython.display import *
connectionParameter = '{"serverName":"localhost,' + sql_port + '","providerName":"MSSQL","authenticationType":"SqlLogin","userName":"sa","password":' + json.dumps(sql_password) + '}'
display(HTML('<br/><a href="command:azdata.connect?' + html.escape(connectionParameter)+'"><font size="3">Click here to connect to the Azure SQL Edge instance</font></a><br/>'))
display(HTML('<br/><span style="color:red"><font size="2">NOTE: The Azure SQL Edge instance password is included in this link, you may want to clear the results of this code cell before saving the notebook.</font></span>'))

### Stop and remove the container

In [ ]:
stop_container_command = f'docker stop {container_name}'
remove_container_command = f'docker rm {container_name}'
display(HTML("Use this link to: <a href=\"command:workbench.action.terminal.focus\">open the terminal window in Azure Data Studio</a> and use the links below to paste the command to the terminal."))
display(HTML("Stop the container: <a href=\"command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22"+stop_container_command.replace(" ","%20")+"%22%7D\">" + stop_container_command + "</a>"))
display(HTML("Remove the container: <a href=\"command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22"+remove_container_command.replace(" ","%20")+"%22%7D\">" + remove_container_command + "</a>"))